In [28]:
!pip install augraphy albumentations tqdm

import pandas as pd 
import numpy as np 

import os 
from glob import glob 

from PIL import Image
import albumentations as A
from albumentations import ImageOnlyTransform
from augraphy import *

from tqdm import tqdm

In [45]:
#잘못된 label 수정
train = pd.read_csv('../data/train.csv')

train['target'][train['ID'] == '45f0d2dfc7e47c03.jpg'] = 7
train['target'][train['ID'] == 'aec62dced7af97cd.jpg'] = 14
train['target'][train['ID'] == '8646f2c3280a4f49.jpg'] = 3
train['target'][train['ID'] == '1ec14a14bbe633db.jpg'] = 7
train['target'][train['ID'] == '7100c5c67aecadc5.jpg'] = 7
train['target'][train['ID'] == 'c5182ab809478f12.jpg'] = 14
train['target'][train['ID'] == '38d1796b6ad99ddd.jpg'] = 6
train['target'][train['ID'] == '0583254a73b48ece.jpg'] = 6
train.to_csv('../data/train.csv', index = False)






In [46]:
df = pd.read_csv('../data/train.csv')
data_path = '../data/train/'

In [47]:
horizontal_flip = A.Compose([
    A.HorizontalFlip(p=1),
])

vertical_flip = A.Compose([
    A.VerticalFlip(p=1),
])

double_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

transpose = A.Compose([
    A.Transpose(p=1)
])

transpose_hflip = A.Compose([
    A.Transpose(p=1), 
    A.HorizontalFlip(p=1),
])

transpose_vflip = A.Compose([
    A.Transpose(p=1),
    A.VerticalFlip(p=1),
])

transpose_dflip = A.Compose([
    A.Transpose(p=1),  
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

pattern = PatternGenerator(
            imgx = 120,
            imgy = 120,
            n_rotation_range = (10, 15),
            alpha_range = (0.1, 0.2),
            color = (130, 130, 130),
        )

class PatternGeneratorTransform(ImageOnlyTransform):
    def __init__(self, pattern_generator, always_apply=False, p=1.0):
        super().__init__(always_apply, p)
        self.pattern_generator = pattern_generator

    def apply(self, img, **params):
        return self.pattern_generator(img)

noise_rotate = A.Compose([
    # A.GaussNoise(var_limit=(100, 1000), per_channel=False, p=1),
    # A.Blur(blur_limit=(2, 4), p=0.5),
    # A.ToGray(p=0.2),
    # A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0, rotate_limit=30, p=0.4, border_mode=0, value=(255, 255, 255)),

    A.OneOf([A.GaussNoise(var_limit=(100, 900), per_channel=False, p=1), PatternGeneratorTransform(pattern, p=1)], p=0.8), 
    A.OneOf([A.MotionBlur(p=0.2), A.MedianBlur(blur_limit=3, p=0.2), A.Blur(blur_limit=(2, 4), p=1),], p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),    
    A.ToGray(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0, rotate_limit=30, p=0.5, border_mode=0, value=[255, 255, 255]),
    A.GaussNoise(var_limit=(10, 50), p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.5, p=0.5),
    A.Blur(blur_limit=3, p=0.5),
    A.MotionBlur(blur_limit=3, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=1, min_height=1, min_width=1, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=0.5),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
    A.ChannelShuffle(p=0.5),
    A.InvertImg(p=0.5),
    A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, p=0.5),
    A.ToGray(p=0.5),
    A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.5),
    A.Perspective(scale=(0.05, 0.1), p=0.5),
    A.Solarize(threshold=128, p=0.5),
    A.OneOf([A.GaussNoise(var_limit=(100, 900), per_channel=False, p=1), PatternGeneratorTransform(pattern, p=1)], p=0.8),
    A.OneOf([A.MotionBlur(p=0.2), A.MedianBlur(blur_limit=3, p=0.2), A.Blur(blur_limit=(2, 4), p=1)], p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),    
    A.ToGray(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0, rotate_limit=30, p=0.5, border_mode=0, value=[255, 255, 255]),
    A.GaussNoise(var_limit=(10, 50), p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.5, p=0.5),
    A.Blur(blur_limit=3, p=0.5),
    A.MotionBlur(blur_limit=3, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=1, min_height=1, min_width=1, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=0.5),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
    A.ChannelShuffle(p=0.5),
    A.InvertImg(p=0.5),
    A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, p=0.5),
    A.ToGray(p=0.5),
    A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.5),
    A.Perspective(scale=(0.05, 0.1), p=0.5),
    A.Solarize(threshold=128, p=0.5)
])
    

In [48]:
base_aug_types = [
    ("hf_", horizontal_flip),
    ("vf_", vertical_flip),
    ("df_", double_flip),
    ("t_", transpose),
    ("thf_", transpose_hflip),
    ("tvf_", transpose_vflip),
    ("tdf_", transpose_dflip)
]

In [50]:
ids = []
targets = []
for index, ID, target in tqdm(df.itertuples(), total=df.shape[0], desc='Image augmentation'):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))
    
    for prefix, aug_function in base_aug_types:
        transformed_image = aug_function(image=image)['image']
        new_ID = prefix + ID
        ids.append(new_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, new_ID))
        
aug_data = {
    'ID' : ids,
    'target' : targets
}
aug_data = pd.DataFrame(aug_data)    
df = pd.concat([df, aug_data])


Image augmentation: 100%|██████████| 1570/1570 [01:09<00:00, 22.55it/s]


In [51]:
ids = []
targets = []
for index, ID, target in tqdm(df.itertuples(), total=df.shape[0], desc='Image augmentation'):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))

    for i in range(7):
        transformed_image = noise_rotate(image=image)['image']
        image_ID = f'nr{i}_' + ID 
        ids.append(image_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))
    
aug_data = {
    'ID' : ids,
    'target' : targets
}
aug_data = pd.DataFrame(aug_data)    
df = pd.concat([df, aug_data])

Image augmentation:   2%|▏         | 189/12560 [05:06<5:34:46,  1.62s/it]


KeyboardInterrupt: 

In [56]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform

# 기존 증강 기법
horizontal_flip = A.Compose([
    A.HorizontalFlip(p=1),
])

vertical_flip = A.Compose([
    A.VerticalFlip(p=1),
])

double_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

transpose = A.Compose([
    A.Transpose(p=1)
])

transpose_hflip = A.Compose([
    A.Transpose(p=1), 
    A.HorizontalFlip(p=1),
])

transpose_vflip = A.Compose([
    A.Transpose(p=1),
    A.VerticalFlip(p=1),
])

transpose_dflip = A.Compose([
    A.Transpose(p=1),  
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

pattern = PatternGenerator(
    imgx=120,
    imgy=120,
    n_rotation_range=(10, 15),
    alpha_range=(0.1, 0.2),
    color=(130, 130, 130),
)

class PatternGeneratorTransform(ImageOnlyTransform):
    def __init__(self, pattern_generator, always_apply=False, p=1.0):
        super().__init__(always_apply, p)
        self.pattern_generator = pattern_generator

    def apply(self, img, **params):
        return self.pattern_generator(img)

# 더 다양한 noise_rotate 증강 기법 추가
noise_rotate = A.Compose([
    A.OneOf([A.GaussNoise(var_limit=(100, 900), per_channel=False, p=1), PatternGeneratorTransform(pattern, p=1)], p=0.8),
    A.OneOf([A.MotionBlur(p=0.2), A.MedianBlur(blur_limit=3, p=0.2), A.Blur(blur_limit=(2, 4), p=1)], p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),    
    A.ToGray(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0, rotate_limit=30, p=0.5, border_mode=0, value=[255, 255, 255]),
    A.GaussNoise(var_limit=(10, 50), p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.5, p=0.5),
    A.Blur(blur_limit=3, p=0.5),
    A.MotionBlur(blur_limit=3, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=1, min_height=1, min_width=1, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=0.5),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
    A.ChannelShuffle(p=0.5),
    A.InvertImg(p=0.5),
    A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, p=0.5),
    A.ToGray(p=0.5),
    A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.5),
    A.Perspective(scale=(0.05, 0.1), p=0.5),
    A.Solarize(threshold=128, p=0.5)
])

# 기본 증강 기법 리스트
base_aug_types = [
    ("hf_", horizontal_flip),
    ("vf_", vertical_flip),
    ("df_", double_flip),
    ("t_", transpose),
    ("thf_", transpose_hflip),
    ("tvf_", transpose_vflip),
    ("tdf_", transpose_dflip)
]

# 증강된 데이터프레임에 대한 리스트
ids = []
targets = []

# 기존 데이터에 기본 증강 적용
for index, ID, target in tqdm(df.itertuples(), total=df.shape[0], desc='Base augmentation'):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))
    
    for prefix, aug_function in base_aug_types:
        transformed_image = aug_function(image=image)['image']
        new_ID = prefix + ID
        ids.append(new_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, new_ID))
        
aug_data = {
    'ID': ids,
    'target': targets
}
aug_data = pd.DataFrame(aug_data)    
df = pd.concat([df, aug_data])


Base augmentation: 100%|██████████| 100480/100480 [1:09:40<00:00, 24.04it/s]


In [57]:
df.to_csv('../data/aug_train.csv', index=False)

In [58]:
df.shape

(803840, 2)

In [59]:
!pip install timm
!pip install wandb
!pip install matplotlib

In [76]:
import os
import time
import random
import copy

import timm
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
import albumentations as A
from albumentations import ImageOnlyTransform
from augraphy import *
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import wandb
import matplotlib.pyplot as plt

In [77]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [78]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None, oversample=False):
        self.df = pd.read_csv(csv)
        self.path = path
        self.transform = transform
        self.oversample = oversample

        # 클래스간 불균형 해소를 위한 샘플 증식
        if self.oversample:
            # 각 클래스별로 데이터 수 계산
            class_counts = np.bincount(self.df.values[:, 1].astype(int))

            # 각 클래스별로 증식할 횟수 설정 (이 예제에서는 최대 데이터 수에 맞춤)
            max_class_count = max(class_counts)
            oversample_factors = [max_class_count // count for count in class_counts]
            # Class 3, 7 가중치 2로 변경
            oversample_factors[3] = 2
            oversample_factors[7] = 2 
            # oversample_factors[14] = 3 

            # 각 클래스별로 데이터를 증식한 새로운 데이터 프레임 생성
            oversampled_data = [self.df.values[self.df.values[:, 1] == cls].repeat(factor, axis=0) for cls, factor in enumerate(oversample_factors)]
            oversampled_data = np.vstack(oversampled_data)

            self.df = pd.DataFrame(oversampled_data, columns=self.df.columns)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df.iloc[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target
    
# meta.csv 파일 읽기
meta_data = pd.read_csv('../data/meta.csv')
label_to_class_name = dict(zip(meta_data['target'], meta_data['class_name']))

In [ ]:

model_name = 'efficientnet_b4'
pretrained_size = 380
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]

# training config
LR = 5e-4
EPOCHS = 40
BATCH_SIZE = 32
dropout_ratio = 0.2
patience = 1
num_workers = 0
num_classes = 100


In [80]:
# train image 변환을 위한 transform 코드
train_transform = A.Compose([
    # PatternGeneratorTransform(pattern, p=0.3), # 패턴 노이즈
    # A.Resize(height=pretrained_size, width=pretrained_size), # 이미지 크기 조정
    # 이미지 긴 측면 크기 조절 후 패딩 적용
    A.LongestMaxSize(max_size=pretrained_size, always_apply=True), 
    A.PadIfNeeded(min_height=pretrained_size, min_width=pretrained_size, border_mode=0, value=(255, 255, 255)),
    
    A.Normalize(mean=pretrained_means, std=pretrained_stds), # images normalization
    ToTensorV2() # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
])

# test image 변환을 위한 transform 코드
test_transform = A.Compose([    
    # A.Resize(height=pretrained_size, width=pretrained_size),
    A.LongestMaxSize(max_size=pretrained_size, always_apply=True),
    A.PadIfNeeded(min_height=pretrained_size, min_width=pretrained_size, border_mode=0, value=(255, 255, 255)),
    
    A.Normalize(mean=pretrained_means, std=pretrained_stds),
    ToTensorV2()
])

aug_test_transform = A.Compose([    
    A.RandomRotate90(),
    A.Flip(p=0.5),              
                        
    # A.Resize(height=pretrained_size, width=pretrained_size),
    A.LongestMaxSize(max_size=pretrained_size, always_apply=True),
    A.PadIfNeeded(min_height=pretrained_size, min_width=pretrained_size, border_mode=0, value=(255, 255, 255)),
    
    A.Normalize(mean=pretrained_means, std=pretrained_stds),
    ToTensorV2()
])

# 시각화를 위한 transform 코드
base_transform = A.Compose([
    ToTensorV2()
])

In [81]:
# Training Dataset 정의
train_dataset = ImageDataset(
    data_path + 'aug_train.csv',
    data_path + 'train/',
    transform=train_transform,
    oversample=True
)

# Test Dataset 정의
test_dataset = ImageDataset(
    data_path + 'sample_submission.csv',
    data_path + 'test/',
    transform=test_transform
)

aug_test_dataset = ImageDataset(
    data_path + 'sample_submission.csv',
    data_path + 'test/',
    transform=aug_test_transform
)

# 시각화용 Dataset 정의
train_dataset_v = ImageDataset(
    data_path + 'aug_train.csv',
    data_path + 'train/',
    transform=base_transform
)

test_dataset_v = ImageDataset(
    data_path + 'sample_submission.csv',
    data_path + 'test/',
    transform=base_transform
)

print(len(train_dataset), len(test_dataset))

955904 3140


In [69]:
# 데이터 셋을 학습 데이터 셋과 검증 데이터 셋으로 분리
total_size = len(train_dataset)
train_num, valid_num = int(total_size * VALID_RATIO), total_size - int(total_size * VALID_RATIO)

# train - valid set 나누기
generator = torch.Generator().manual_seed(SEED)
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_num, valid_num], generator = generator)

valid_data = copy.deepcopy(valid_dataset)
valid_data.dataset.transform = test_transform

print(f'Train dataset 개수: {len(train_dataset)}')
print(f'Validation dataset 개수: {len(valid_dataset)}')
print(f'Test dataset 개수: {len(test_dataset)}')

Train dataset 개수: 764723
Validation dataset 개수: 191181
Test dataset 개수: 3140


In [82]:
# DataLoader 정의
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
    )

valid_dataloader = DataLoader(
    valid_dataset, 
    batch_size = BATCH_SIZE, 
    shuffle = False,
    num_workers=0,
    pin_memory=True
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
    )

aug_test_dataloader = DataLoader(
    aug_test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
    )

In [83]:
# 모델 생성
model = CustomEfficientNetB4(num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

model

CustomEfficientNetB4(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, 

In [103]:
run = wandb.init(project = 'newwhy', name = 'training')

# wandb에 모델의 weight & bias, graident 시각화
# run.watch(model, loss_fn, log = 'all', log_graph = True)

model, valid_max_accuracy = training_loop(model, train_dataloader, valid_dataloader, loss_fn, optimizer, device, EPOCHS, patience, model_name, run)
print(f'Valid Max Accuracy: {valid_max_accuracy}')

KeyboardInterrupt: 

In [ ]:
# run.unwatch()
run.finish()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.load_state_dict(torch.load(f'./model_{model_name}.pt'))
model.to(device)

N_TTA = 20
preds_list = []
with torch.no_grad():
    loaders = [test_dataloader] + [aug_test_dataloader] * N_TTA

    for batches in tqdm(zip(*loaders), total=len(test_dataloader)):
        images, *aug_images = [images.to(device) for images, _ in batches]

        outputs_original = model(images)
        outputs_augmented = [model(aug_image) for aug_image in aug_images]

        final_outputs = (outputs_original + sum(outputs_augmented)) / N_TTA + 1
        preds_list.extend(final_outputs.argmax(dim=1).cpu().numpy())

# 예측 결과 확인
print("Ensemble Predictions:", preds_list)

In [ ]:
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [ ]:
sample_submission_df = pd.read_csv(data_path + 'sample_submission.csv')
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [ ]:
pred_df.to_csv("pred.csv", index=False)

In [ ]:
pred_df.head()

In [92]:
# 예측 파일 불러오기
pred1 = pd.read_csv("9180.csv")
pred2 = pd.read_csv("9027.csv")
pred3 = pd.read_csv("8600.csv")
pred4 = pd.read_csv("8519.csv")
pred5 = pd.read_csv("7266.csv")

# Combine predictions into a DataFrame
ensemble_df = pd.DataFrame({
    'ID': pred1['ID'],  # Assuming 'ID' column exists in all prediction files
    'pred1': pred1['target'],
    'pred2': pred2['target'],
    'pred3': pred3['target'],
    'pred4': pred4['target'],
    'pred5': pred5['target'],
})

# Perform majority voting to get the final prediction
ensemble_df['target'] = ensemble_df[['pred1', 'pred3', 'pred3','pred4','pred5']].mode(axis=1)[0].astype(int)

sample_submission_df = pd.read_csv(data_path + 'sample_submission.csv')
assert (sample_submission_df['ID'] == ensemble_df['ID']).all()

# Save the final predictions
ensemble_df[['ID', 'target']].to_csv('ensemble_v2.csv', index=False)

In [93]:
import pandas as pd
from collections import Counter

# CSV 파일 불러오기
pred1 = pd.read_csv("9180.csv")
pred2 = pd.read_csv("9027.csv")
pred3 = pd.read_csv("8600.csv")
pred4 = pd.read_csv("8519.csv")
pred5 = pd.read_csv("7266.csv")




# 각 예측 결과를 리스트로 저장
predictions = [pred1['target'], pred2['target'], pred3['target'], pred4['target'], pred5['target']]

# 투표를 통해 최종 예측 결정
final_predictions = []

# 각 샘플에 대해 투표
for preds in zip(*predictions):
    vote_result = Counter(preds).most_common(1)[0][0]
    final_predictions.append(vote_result)

# 최종 예측 결과를 데이터프레임으로 저장
submission = pd.DataFrame({'ID': pred1['ID'], 'target': final_predictions})

# 결과를 CSV 파일로 저장
submission.to_csv("ensemble_voting_submission.csv", index=False)


In [104]:
import pandas as pd
from collections import Counter

# CSV 파일 불러오기
pred1 = pd.read_csv("9180.csv")
pred2 = pd.read_csv("9027.csv")
pred3 = pd.read_csv("8600.csv")
pred4 = pd.read_csv("8519.csv")
pred5 = pd.read_csv("7266.csv")
pred6 = pd.read_csv("9113.csv")
pred7 = pd.read_csv("9188.csv")
pred8 = pd.read_csv("6301.csv")

# 각 예측 결과를 리스트로 저장
predictions = [pred1['target'], pred2['target'], pred3['target'], pred4['target'], pred5['target'], pred6['target'], pred7['target'], pred8['target']]

# 투표를 통해 최종 예측 결정
final_predictions = []

# 각 샘플에 대해 투표
for preds in zip(*predictions):
    vote_result = Counter(preds).most_common(1)[0][0]
    final_predictions.append(vote_result)

# 최종 예측 결과를 데이터프레임으로 저장
submission = pd.DataFrame({'ID': pred1['ID'], 'target': final_predictions})

# 결과를 CSV 파일로 저장
submission.to_csv("ensemble_voting_submission2.csv", index=False)
